In [ ]:
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch
from huggingface_hub import login
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


with open("HF_TOKEN.txt", "r") as f:
    token = f.read().strip()

login(token=token)

In [ ]:
#Load data
df = pd.read_parquet('./data/Silver/Cleaned_data.parquet')


#df = df.sample(n=1000).reset_index()
# df = pd.read_parquet('./data/Silver/data_for_trainning_encoder.parquet')
df

In [ ]:
model_name = "google/t5gemma-9b-9b-ul2-it"

## Load the model T5-gemma
# Configuración de la cuantificación de 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                 
    bnb_4bit_quant_type="nf4",        
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_use_double_quant=True,   
)


## al tener dos GPU de 12 G tengo que dividir manualmente el modelo porque el "auto" lo fragmenta
# y a la hora de generar el output genera error
device_map = {
    'model.encoder': 'cuda:0',
    'model.decoder': 'cuda:1',
    'lm_head': 'cuda:1',
    'model.shared': 'cuda:0' 
}


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map=device_map, 
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
)

In [ ]:
query = """DETECTA si el texto contiene información personal explícita, como un número de teléfono, una dirección postal completa o un número de DNI, información de carácter personal con el que puedan reconocer a la persona. Responde solo con 'SI' o 'NO'."""

def create_prompt(documento):
    return [{
        "role": "user",
        "content": (
            "Eres un analista experto en mensajes breves. Tu única tarea es extraer información explícita del mensaje.\n"
            "Responde **solo** con el valor solicitado. No añadas introducciones, explicaciones, encabezados, ni texto adicional.\n"
            "No uses formato, como negritas, cursivas o cualquier otro tipo de markdown.\n"
            "Utiliza el formato específico de la pregunta (ej. 'SI'/'NO').\n"
            "Responde siempre en español.\n"
            f"Tarea: {query}\n"
            f"Texto: {documento}"
        )
    }]

def process_batch(batch_texts, batch_size=4):  # Ajusta el batch_size según tu memoria GPU
    """Procesa un batch de textos"""
    # Crear prompts para todo el batch
    batch_prompts = [create_prompt(texto) for texto in batch_texts]
    
    # Tokenizar todo el batch
    batch_inputs = []
    for prompt in batch_prompts:
        tokenized = tokenizer.apply_chat_template(
            prompt, 
            return_tensors="pt", 
            return_dict=True, 
            add_generation_prompt=True,
            padding=False  # No hacer padding aquí
        )
        batch_inputs.append(tokenized)
    
    # Extraer input_ids y attention_masks
    input_ids_list = [inputs['input_ids'].squeeze(0) for inputs in batch_inputs]
    attention_mask_list = [inputs['attention_mask'].squeeze(0) for inputs in batch_inputs]
    
    # Hacer padding manual para que todos tengan la misma longitud
    max_length = max(len(ids) for ids in input_ids_list)
    
    padded_input_ids = []
    padded_attention_masks = []
    
    for i, (ids, mask) in enumerate(zip(input_ids_list, attention_mask_list)):
        padding_length = max_length - len(ids)
        if padding_length > 0:
            # Padding a la izquierda para modelos generativos
            padded_ids = torch.cat([
                torch.full((padding_length,), tokenizer.pad_token_id, dtype=ids.dtype),
                ids
            ])
            padded_mask = torch.cat([
                torch.zeros(padding_length, dtype=mask.dtype),
                mask
            ])
        else:
            padded_ids = ids
            padded_mask = mask
            
        padded_input_ids.append(padded_ids)
        padded_attention_masks.append(padded_mask)
    
    # Convertir a tensores
    batch_input_ids = torch.stack(padded_input_ids).to('cuda:0')
    batch_attention_masks = torch.stack(padded_attention_masks).to('cuda:0')
    
    # Generar respuestas para todo el batch
    with torch.no_grad():
        outputs = model.generate(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_masks,
            max_new_tokens=4,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=False,  # No usar cache para ahorrar memoria
        )
    del batch_input_ids, batch_attention_masks
    torch.cuda.empty_cache()
    
    # Decodificar respuestas
    responses = []
    for output in outputs:
        response = tokenizer.decode(output, skip_special_tokens=True)
        responses.append(response.strip())
        
    del outputs
    torch.cuda.empty_cache()
    
    return responses

In [ ]:
# Procesamiento en batches
LOPD = []
batch_size = 64 # Ajusta según tu memoria GPU disponible

# Convertir DataFrame a lista para facilitar el batching
tweets_list = df["tweets"].tolist()

# Procesar en batches
for i in tqdm(range(0, len(tweets_list), batch_size), desc="Processing Batches"):
    batch_texts = tweets_list[i:i+batch_size]
    batch_responses = process_batch(batch_texts, batch_size)
    LOPD.extend(batch_responses)

print(f"Procesados {len(LOPD)} tweets")


In [ ]:
df['proteccion_datos'] = LOPD
df['proteccion_datos'].value_counts()
#hay malas salidas, hay que limpiar.

In [ ]:
#arreglar valores extraños. 
df['proteccion_datos'] = df['proteccion_datos'].str.upper().astype(str).str.strip()
df['proteccion_datos'] = df['proteccion_datos'].str.replace('1. NO', 'NO')
df['proteccion_datos'] = df['proteccion_datos'].str.replace('NO.', 'NO')
df['proteccion_datos'].value_counts()

In [ ]:
df[df['proteccion_datos'].str.upper().str.strip() == 'SI']

In [ ]:
for idx , i in df['tweets'][df['proteccion_datos'].str.upper().str.strip() == 'SI'].items():
    print(idx,'==', i)


In [ ]:
df = df[df['proteccion_datos'] == 'NO']
df = df[['User', 'tweets', 'search','fecha_captura']]
df

In [ ]:
#Se puede observar que hay bastantes falsos positivos. Sin embargo, son mensajes que poco o nada aportan al analisis, 
#muchos de ellos hablan sobre que se les ha pedido el DNI.  Habría que refinar más el prompt, pero vale como criba inicial. 

In [ ]:
df.to_parquet( './data/Silver/Cleaned_data_LOPD.parquet' ,index=False)